In [1]:
import time
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from mxnet import nd, autograd, gluon
from mxnet.gluon import nn, rnn
import mxnet as mx
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.decomposition import PCA
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv(r'../.././Documents/data/TRAINSET_STOCK.csv')
df['vol_change'] = df.groupby(["ts_code"])['vol'].pct_change()
df['vol_change'] = df['vol_change'].fillna(0)
df.head()

,ts_code,trade_date,name,open,low,high,close,change,pct_change,vol,amount,pe,pb,y,vol_change
0,801010,20140401,农林牧渔,1668.75,1668.54,1689.12,1689.07,22.13,1.33,34914.0,291113.0,41.51,2.77,1,0.000000
1,801010,20140402,农林牧渔,1688.72,1684.53,1693.41,1692.24,3.17,0.19,36300.0,289020.0,41.63,2.79,1,0.039698
2,801010,20140403,农林牧渔,1693.05,1679.85,1697.73,1685.71,-6.53,-0.39,31403.0,259464.0,41.38,2.78,0,-0.134904
3,801010,20140404,农林牧渔,1681.92,1680.34,1698.44,1698.25,12.54,0.74,28648.0,240940.0,41.76,2.80,1,-0.087730
4,801010,20140408,农林牧渔,1693.24,1692.22,1706.84,1706.84,8.59,0.51,35012.0,312423.0,42.00,2.79,1,0.222145


# LSTM

In [4]:
df1 = df[['ts_code','trade_date','name','pct_change','vol_change','y']]
test_df = df.groupby(['trade_date', 'ts_code'])['pct_change']
df2 = df.loc[df1['name']== '农林牧渔']
def creat_dataset(dataset,look_back=10):
    dataX,dataY = [],[]
    for i in range(len(dataset)-look_back):
        a = dataset[i:(i+look_back)]
        dataX.append(a)
        dataY.append(dataset[i+look_back])
    return np.array(dataX,np.array(dataY))